 创建模版与创建响应格式

In [1]:
# 1. 创建提示模板
from langchain.prompts import PromptTemplate
prompt_template = """您是一位专业的鲜花店文案撰写员。
对于售价为 {price} 元的 {flower_name} ，您能提供一个吸引人的简短描述吗？
{format_instructions}"""


# 定义我们想要接收的响应模式
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
response_schemas = [
    ResponseSchema(name="description", description="鲜花的描述文案"),
    ResponseSchema(name="reason", description="问什么要这样写这个文案")
]
# 创建输出解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 获取格式指示
format_instructions = output_parser.get_format_instructions()
# 根据模板创建提示，同时在提示中加入输出解析器的说明
prompt = PromptTemplate.from_template(prompt_template, 
                partial_variables={"format_instructions": format_instructions}) 

print(prompt)

input_variables=['flower_name', 'price'] partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"description": string  // 鲜花的描述文案\n\t"reason": string  // 问什么要这样写这个文案\n}\n```'} template='您是一位专业的鲜花店文案撰写员。\n对于售价为 {price} 元的 {flower_name} ，您能提供一个吸引人的简短描述吗？\n{format_instructions}'


In [4]:
flowers = ["玫瑰", "百合", "康乃馨"]
prices = ["50", "30", "20"]

from langchain_openai import OpenAI
model = OpenAI(model_name='gpt-3.5-turbo-instruct')

# 创建一个空的DataFrame用于存储结果
import pandas as pd
df = pd.DataFrame(columns=["flower", "price", "description", "reason"]) # 先声明列名

for flower, price in zip(flowers, prices):
    input = prompt.format(flower_name=flower, price=price)
    print("---")
    print(input)
    print("---")
    output = model.invoke(input)
    print(output)
    parsed_output = output_parser.parse(output) # 解析大模型输出，返回字典
    parsed_output['flower'] = flower  # 添加变量字段
    parsed_output['price'] = price
    df.loc[len(df)] = parsed_output  # 字典放到list中即：DataFrame
print(df.to_dict(orient='records'))
df.to_csv("flowers_with_descriptions.csv", index=False) # 保存DataFrame到CSV文件

---
您是一位专业的鲜花店文案撰写员。
对于售价为 50 元的 玫瑰 ，您能提供一个吸引人的简短描述吗？
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"description": string  // 鲜花的描述文案
	"reason": string  // 问什么要这样写这个文案
}
```
---
json
{
	"description": "这束粉色的玫瑰散发着浓郁的花香，每一朵都像是爱的告白，温柔而又充满力量。它是表达爱意、感谢、祝福的完美选择。",
	"reason": "这样的文案可以吸引顾客的注意力，让他们感受到玫瑰的浪漫和温馨，同时也突出了玫瑰作为送礼的完美选择。" 
}


OutputParserException: Got invalid JSON object. Error: Expecting value: line 1 column 1 (char 0)